In [10]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.spatial.transform import Rotation
import cicp.cicp_util as CU
import os 

# geometry = "CROSS ROUNDED" # "HDMI_PC" or "EXTRUSION" 
geometry = "BNC" # "HDMI_PC" or "EXTRUSION" 

# base_path = f"C:/Users/NegiA/Desktop/PIH_RSS/cicp/data/KUKA_ICP_DATA/{geometry}/map_and_obs_csv_with_fk/"
# base_path = f"C:/Users/NegiA/Desktop/PIH_RSS/cicp/data/KUKA_ICP_DATA/{geometry}/map_and_obs_csvs_with_fk/"

file_type = "MAP" # "MAP" or "OBS" 
file_num = "1"  

# base_path = "C:/Users/NegiA/Desktop/PIH_RSS/cicp/data/RSS/CROSS/"
base_path = "/home/rp/abhay_ws/contact-manifold-state-estimation/real_data/"
# all_files = os.listdir(dir)
# all_files = [f for f in all_files if f.endswith('.npy')]
# all_files = [f for f in all_files if "T_B_H" not in f] 

# map_files = [
#     f"dhanush_{geometry}_MAP1_moveit_fk.csv",
#     f"dhanush_{geometry}_MAP2_moveit_fk.csv",
#     f"dhanush_{geometry}_MAP3_moveit_fk.csv"
# ]
# obs_files = [
#     f"dhanush_{geometry}_OBS1_moveit_fk.csv",
#     f"dhanush_{geometry}_OBS2_moveit_fk.csv",
#     f"dhanush_{geometry}_OBS3_moveit_fk.csv",
#     f"dhanush_{geometry}_OBS4_moveit_fk.csv",
#     f"dhanush_{geometry}_OBS5_moveit_fk.csv",
# ]

# all_files = [
#     f"dhanush_{geometry}_{file_type}{file_num}_moveit_fk.csv",
#     # f"dhanush_{geometry}_{file_type}{1}_moveit_fk.csv",
#     # f"dhanush_{geometry}_{file_type}{2}_moveit_fk.csv",
#     # f"dhanush_{geometry}_{file_type}{3}_moveit_fk.csv",
#     # f"dhanush_{geometry}_{file_type}{4}_moveit_fk.csv",
#     # f"dhanush_{geometry}_{file_type}{5}_moveit_fk.csv",
#     # f"Trial_{file_num}.csv"
# ]

all_files = [
    # "observations_spiral_tilt_0_52328.242425766.npy", 
    # "observations_spiral_tilt_0_52609.254414758.npy", 
    # "observations_spiral_tilt_0_62624.119995464.npy", 
    # "observations_spiral_tilt_0_6995.55789312.npy", 
    "observations_spiral_tilt_0_9702.322980436.npy", 
]

all_files = ["BNC_ideal_trajectory_forward_processed.csv"] 

# saved_file_type = ".npy" # ".csv" or ".npy" 
saved_file_type = ".csv" # ".csv" or ".npy" 

# Read and combine all maps
if saved_file_type == ".csv":
    map_dfs = [pd.read_csv(base_path + file) for file in all_files]
elif saved_file_type == ".npy":
    map_dfs = [pd.DataFrame(np.load(base_path + file, allow_pickle=True), columns=['FK_X','FK_Y','FK_Z','FK_A','FK_B','FK_C']) for file in all_files]
# map_dfs = [pd.read_csv(base_path + file) for file in map_files]
# map_dfs = [pd.read_csv(base_path + file) for file in obs_files]
combined_map_df = pd.concat(map_dfs, axis=0, ignore_index=True)

# Optional: verify the shape
print(f"Combined shape: {combined_map_df.shape}")

Combined shape: (8500, 33)


In [11]:
column_mapping = {
    'FK_X': 'x',
    'FK_Y': 'y',
    'FK_Z': 'z',
    # 'FK_QX': 'qx',
    # 'FK_QY': 'qy',
    # 'FK_QZ': 'qz',
    # 'FK_QW': 'qw',
    'FK_A': 'a',
    'FK_B': 'b',
    'FK_C': 'c',
    # 'cartForce1_X': 'FX',
    # 'cartForce1_Y': 'FY',
    # 'cartForce1_Z': 'FZ',
    # 'cartTorque1_TauX': 'TX',
    # 'cartTorque1_TauY': 'TY',
    # 'cartTorque1_TauZ': 'TZ',
}

# Rename columns
combined_map_df = combined_map_df[list(column_mapping.keys())].rename(columns=column_mapping)

In [12]:
combined_map_df.describe() 

,x,y,z,a,b,c
count,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000
mean,718.829761,-43.221940,213.739575,148.359617,-0.858288,171.163967
std,0.996239,0.791074,10.493942,33.028531,1.327248,45.423761
min,716.917743,-45.636765,201.925594,78.386773,-3.650778,-179.996242
25%,718.070352,-43.598981,206.170854,164.943882,-0.665896,176.399048
50%,718.405366,-43.379131,206.206991,165.259084,-0.543740,176.514953
75%,719.813661,-43.061996,226.784955,165.902910,0.153300,176.652182
max,721.923798,-41.684743,227.269423,166.824351,0.597303,179.997379


In [13]:
# define peg and hole transformations 

match geometry: 
    case "EXTRUSION": 
        pose_insert = CU.create_pose_euler(
            # x=691.15, y=-109.44, z=197.48, rz=-179.12, ry=1.51, rx=-179.07
            x=691.62, y=-111.64, z=197.35, rz=+179.36, ry=0.95, rx=-179.62 # 1/4/25
        )
        pose_tool_wrt_flange = CU.batch_poses_xyzquat_to_poses_xyzabc(
            CU.create_pose_quaternion(x=0, y=0, z=110.0, qx=0, qy=1, qz=0, qw=0)
        )
        pose_peg_wrt_flange = CU.create_pose_euler(x=0, y=0, z=85.0, rz=0, ry=180, rx=0)
    case "HDMI_PC":
        pose_insert = CU.create_pose_euler(
            x=509.14, y=-52.68, z=441.90, rz=-139.43, ry=0.43, rx=-178.49
        )
        pose_tool_wrt_flange = CU.batch_poses_xyzquat_to_poses_xyzabc(
            CU.create_pose_quaternion(x=0, y=0, z=107.0, qx=0, qy=1, qz=0, qw=0)
        )
        pose_peg_wrt_flange = CU.create_pose_euler(x=0, y=0, z=98.5, rz=0, ry=180, rx=0)
    case "CROSS": 
        pose_insert = CU.create_pose_euler(
            x=694.93, y=88.11, z=195.51, rz=-178.99, ry=0.56, rx=-179.52
        )
        pose_tool_wrt_flange = CU.batch_poses_xyzquat_to_poses_xyzabc(
            CU.create_pose_quaternion(x=0, y=0, z=110.0, qx=0, qy=1, qz=0, qw=0)
        )
        pose_peg_wrt_flange = CU.create_pose_euler(x=0, y=0, z=85.0, rz=0, ry=180, rx=0)
    case "CROSS ROUNDED": 
        pose_insert = CU.create_pose_euler(
            x=692.59, y=-36.94, z=196.46, rz=179.23, ry=1.40, rx=-179.74
        )
        pose_tool_wrt_flange = CU.batch_poses_xyzquat_to_poses_xyzabc(
            CU.create_pose_quaternion(x=0, y=0, z=110.0, qx=0, qy=1, qz=0, qw=0)
        )
        pose_peg_wrt_flange = CU.create_pose_euler(x=0, y=0, z=85.0, rz=0, ry=180, rx=0)
    case "BNC": 
        pose_insert = CU.create_pose_euler(
            x=718.089591, y=-41.785233, z=201.984210, rz=78.921215, ry=-3.352434, rx=179.542323
        )
        pose_tool_wrt_flange = CU.batch_poses_xyzquat_to_poses_xyzabc(
            CU.create_pose_quaternion(x=0, y=0, z=140.0, qx=0, qy=1, qz=0, qw=0)
        )
        pose_peg_wrt_flange = CU.create_pose_euler(x=0, y=0, z=140.0, rz=0, ry=180, rx=0)

homo_T_F_T = CU.batch_poses_xyzabc_to_matrices(pose_tool_wrt_flange)
homo_T_F_P = CU.batch_poses_xyzabc_to_matrices(pose_peg_wrt_flange)

homo_T_T_F = CU.batch_invert_homogenous_transforms(homo_T_F_T)
homo_T_P_F = CU.batch_invert_homogenous_transforms(homo_T_F_P)

homo_T_B_F0 = CU.batch_poses_xyzabc_to_matrices(pose_insert)
homo_T_F0_B = CU.batch_invert_homogenous_transforms(homo_T_B_F0)

homo_T_T0_B = CU.batch_compose_homogenous_transforms(homo_T_T_F, homo_T_F0_B)
homo_T_P0_B = CU.batch_compose_homogenous_transforms(homo_T_P_F, homo_T_F0_B)

In [14]:
# Get all pose data at once as a single array
poses_data = combined_map_df[["x", "y", "z", "a", "b", "c"]].values

# Convert all poses to matrices in one batch operation
homo_T_B_Fi = CU.batch_poses_xyzabc_to_matrices(poses_data)

# Perform batch transformations
homo_T_T0_Ti = CU.batch_compose_homogenous_transforms(
    CU.batch_compose_homogenous_transforms(
        np.repeat(homo_T_T0_B, homo_T_B_Fi.shape[0], axis=0), homo_T_B_Fi
    ),
    np.repeat(homo_T_F_T, homo_T_B_Fi.shape[0], axis=0),
)

homo_T_P0_Pi = CU.batch_compose_homogenous_transforms(
    CU.batch_compose_homogenous_transforms(
        np.repeat(homo_T_P0_B, homo_T_B_Fi.shape[0], axis=0), homo_T_B_Fi
    ),
    np.repeat(homo_T_F_P, homo_T_B_Fi.shape[0], axis=0),
    # np.repeat(np.eye(4).reshape(1,4,4), homo_T_B_Fi.shape[0], axis=0), # FIXME: this is a quick fix when data is T_B_P
)

# Convert matrices back to poses in one batch
result_poses_tool = CU.batch_matrices_to_poses_xyzabc(homo_T_T0_Ti)
result_poses_peg = CU.batch_matrices_to_poses_xyzabc(homo_T_P0_Pi)

# Create result dataframe with transformed poses
df_tool = pd.DataFrame(result_poses_tool, columns=["x", "y", "z", "a", "b", "c"]) 
df_peg = pd.DataFrame(result_poses_peg, columns=["x", "y", "z", "a", "b", "c"]) # NOTE: use this df (peg wrt hole) 

In [8]:
# # transform wrench data from base frame to peg frame
# # NOTE: this only rotates the wrench data but does not factor torque due to force on moment arm 
# df_peg[['FX','FY','FZ','TX','TY','TZ']] = np.zeros((len(df_peg),6))
# for index, row in df_peg.iterrows():
#     cart_force = combined_map_df.loc[index,['FX','FY','FZ']].values 
#     cart_torque = combined_map_df.loc[index,['TX','TY','TZ']].values  
#     r = Rotation.from_euler('xyz', row[['c','b','a']].values, degrees=True) 
#     peg_force = r.apply(cart_force)
#     peg_torque = r.apply(cart_torque)
#     df_peg.loc[index, 'FX'] = peg_force[0]
#     df_peg.loc[index, 'FY'] = peg_force[1]
#     df_peg.loc[index, 'FZ'] = peg_force[2]
#     df_peg.loc[index, 'TX'] = peg_torque[0]
#     df_peg.loc[index, 'TY'] = peg_torque[1]
#     df_peg.loc[index, 'TZ'] = peg_torque[2] 

In [ ]:
# use log map to compute rotation vector 
def rotation_matrix_to_log_vector(R):
    """Convert a rotation matrix to a rotation vector using the logarithm map."""
    theta = np.arccos((np.trace(R) - 1) / 2)
    if theta < 1e-6:
        return np.zeros(3)
    else:
        ln_R = (theta / (2 * np.sin(theta))) * (R - R.T)
        return np.array([ln_R[2, 1], ln_R[0, 2], ln_R[1, 0]])
    
df_peg[['wx','wy','wz']] = np.zeros((len(df_peg),3))
for index, row in df_peg.iterrows():
    r = Rotation.from_euler('xyz', row[['c','b','a']].values, degrees=True) 
    R = r.as_matrix()
    log_vec = rotation_matrix_to_log_vector(R)
    df_peg.loc[index, 'wx'] = log_vec[0]
    df_peg.loc[index, 'wy'] = log_vec[1]
    df_peg.loc[index, 'wz'] = log_vec[2]
    
df_peg.describe()

In [17]:
# save as csv and pkl files 
# df_peg.to_csv(base_path+"hdmi_real_map.csv", index=False) 
# df_peg.to_pickle(base_path+"hdmi_real_map.pkl") 
# df_peg.to_csv(base_path+"extrusion_real_obs_5.csv", index=False) 
# df_peg.to_pickle(base_path+"extrusion_real_obs_5.pkl") 

# df_peg.to_csv(base_path+f"{geometry}_real_{file_type}_{file_num}_with_wrench.csv", index=False)
# df_peg.to_csv(base_path+f"{geometry}_real_{file_type}_{file_num}.csv", index=False)  
df_peg.to_csv(base_path+f"{geometry}_ideal_trajectory_forward_pose_H_P.csv", index=False)  

In [12]:
error to stop here 

SyntaxError: invalid syntax (887534964.py, line 1)

In [ ]:
CU.plot_map(df_peg)

In [ ]:
def plot_histograms(dataframe: pd.DataFrame):
    # Create a figure with 2x3 subplots (better layout for 6 dimensions)
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Histograms of Poses', fontsize=16)

    # Flatten axes for easier iteration
    axes = axes.flatten()

    # Create histograms for each dimension
    for idx, col in enumerate(["x", "y", "z", "a", "b", "c"]):
        axes[idx].hist(dataframe[col], bins=50, edgecolor='black')
        axes[idx].set_title(f'{col} Distribution')
        axes[idx].set_xlabel(f'{col} value')
        axes[idx].set_ylabel('Frequency')
        
        # Add mean and std as text
        mean = dataframe[col].mean()
        std = dataframe[col].std()
        axes[idx].text(0.05, 0.95, f'Mean: {mean:.3f}\nStd: {std:.3f}', 
                    transform=axes[idx].transAxes,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # Adjust layout to prevent overlap
    plt.tight_layout()

In [ ]:
plot_histograms(df_tool)

In [ ]:
plot_histograms(df_peg)

# Filtering using corners (not needed for now)

In [ ]:
# filter data by extrema points
peg_length = 25.0 
peg_width = 25.0 
half_peg_width = peg_width/2  
extrema_points_coords_peg_frame = [
    np.array([+half_peg_width, +half_peg_width, -peg_length]),
    np.array([-half_peg_width, +half_peg_width, -peg_length]),
    np.array([-half_peg_width, -half_peg_width, -peg_length]),
    np.array([+half_peg_width, -half_peg_width, -peg_length]),
] 

idx_below_top_surface = [] 
z_top_surface_thresh = 0.1  
for index, row in df_combined.iterrows(): 
    T_H_P = pose_to_T(row[['X','Y','Z','A','B','C']].values) 
    for i, extrema_point in enumerate(extrema_points_coords_peg_frame): 
        extrema_points_coords_hole_frame = T_H_P @ np.vstack((extrema_point.reshape(3,1), np.array([[1]]))) # T_HE = T_HP * T_PE 
        z_extrema_point = extrema_points_coords_hole_frame[2,0]
        if z_extrema_point < z_top_surface_thresh: 
            idx_below_top_surface.append(index) 
            break  

print(len(idx_below_top_surface)) 
print(len(df_combined)) 

# Saving filtered Data

In [ ]:
# filter data and output 
df_combined_filtered = df_combined.iloc[idx_below_top_surface,:] 
df_combined_filtered.reset_index(drop=True, inplace=True) 
# save filtered data to csv 
file_path = f"/home/aero/abhay_ws/contact_maps/{geometry}_real_obs.csv" 
df_combined_filtered.to_csv(file_path) 

# save df as pkl file 
file_path = f"/home/aero/abhay_ws/contact_maps/{geometry}_real_obs_df.pkl" 
df_combined_filtered.to_pickle(file_path) 

# save np array as pkl file
file_path = f"/home/aero/abhay_ws/contact_maps/{geometry}_real_obs_np.pkl" 
np.save(file_path, df_combined_filtered.to_numpy()) 

In [ ]:
df_combined_filtered.max() 